In [1]:
print("hello worlds")

hello worlds


In [4]:
import open3d as o3d
print(o3d.__version__)

0.10.0.0


In [3]:
import MinkowskiEngine as ME
print(ME.__version__)

0.5.4


In [8]:
import torch
print(torch.__version__)
print("cuda enable: ", torch.cuda.is_available())
print("cuda device: ", torch.cuda.device_count())

1.10.1
cuda enable:  True
cuda device:  2


In [45]:
import open3d as o3d
import MinkowskiEngine as ME
import torch
import numpy as np

def preprocess(pcd):
    '''
    Stage 0: preprocess raw input point cloud
    Input: raw point cloud
    Output: voxelized point cloud with
    - xyz:    unique point cloud with one point per voxel
    - coords: coords after voxelization
    - feats:  dummy feature placeholder for general sparse convolution
    '''
    if isinstance(pcd, o3d.geometry.PointCloud):
        xyz = np.array(pcd.points)
    elif isinstance(pcd, np.ndarray):
        xyz = pcd
    else:
        raise Exception('Unrecognized pcd type')

    # Voxelization:
    # Maintain double type for xyz to improve numerical accuracy in quantization
    sel = ME.utils.sparse_quantize(
        coordinates=xyz,
        quantization_size=0.05,
        return_index=True)
    npts = len(sel[1])
    xyz = torch.from_numpy(xyz[sel[1]])

    # ME standard batch coordinates
    coords = ME.utils.batched_coordinates(
        [torch.floor(xyz / 0.05).int()])
    feats = torch.ones(npts, 1)

    return xyz.float(), coords, feats

pcd = o3d.io.read_point_cloud("redkitchen_000.ply")
pcd.estimate_normals()

xyz, coords, feats = preprocess(pcd)
# print(xyz)
print("coord: {t}\n".format(t=type(coords)), coords)
print("feats: {t}\n".format(t=type(feats)), feats)

Format = auto
Extension = ply
coord: <class 'torch.Tensor'>
 tensor([[  0,   0,   4,  16],
        [  0,   1,   4,  16],
        [  0,   2,   4,  16],
        ...,
        [  0,  -9, -27,  69],
        [  0,  -8, -27,  69],
        [  0, -10, -26,  69]], dtype=torch.int32)
feats: <class 'torch.Tensor'>
 tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])


In [49]:
# output of demo, transformation matrix T
# [[ 0.99972518  0.01974154  0.0126477  -0.70827058]
#  [-0.02042991  0.99817484  0.05683022  0.18280375]
#  [-0.0115027  -0.05707298  0.99830381 -0.18114951]
#  [ 0.          0.          0.          1.        ]]

T = np.array(
    [
    [ 0.99972518,  0.01974154,  0.0126477,  -0.70827058],
    [-0.02042991,  0.99817484,  0.05683022,  0.18280375],
    [-0.0115027,  -0.05707298,  0.99830381, -0.18114951],
    [ 0.,          0.,          0.,          1.,        ]
    ]
)

pcd.transform(T)
o3d.io.write_point_cloud("demo_output.ply", pcd)

True